In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import pickle
from datetime import datetime

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Crear directorio para plots si no existe
Path("results/plots").mkdir(parents=True, exist_ok=True)

## 1. Cargar Resultados

Carga todos los archivos CSV de resultados generados por los experimentos.

In [ ]:
def load_all_results(results_dir="results"):
    """Carga todos los archivos CSV de resultados"""
    csv_files = glob.glob(f"{results_dir}/*.csv")
    
    if not csv_files:
        print("⚠️ No se encontraron archivos de resultados.")
        return None
    
    print(f"📁 Archivos encontrados: {len(csv_files)}")
    for f in csv_files:
        print(f"  - {Path(f).name}")
    
    # Cargar y combinar todos los DataFrames
    dfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        df['source_file'] = Path(file).stem
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"\n✓ Total de partidas cargadas: {len(combined_df)}")
    print(f"✓ Agentes únicos: {combined_df['agent_name'].nunique()}")
    
    return combined_df

# Cargar datos
df_all = load_all_results()

if df_all is not None:
    print("\n" + "="*60)
    print("VISTA PREVIA DE LOS DATOS")
    print("="*60)
    display(df_all.head())
    print("\nColumnas disponibles:", list(df_all.columns))

## 2. Estadísticas Generales

Resumen estadístico de todos los agentes.

In [ ]:
if df_all is not None:
    # Agrupar por agente
    stats = df_all.groupby('agent_name').agg({
        'max_tile': ['mean', 'std', 'max', 'min'],
        'final_score': ['mean', 'std', 'max'],
        'moves': ['mean', 'std'],
        'time_seconds': ['mean', 'sum'],
        'game_id': 'count'  # Número de partidas
    }).round(2)
    
    stats.columns = ['_'.join(col).strip() for col in stats.columns.values]
    stats = stats.rename(columns={'game_id_count': 'num_games'})
    stats = stats.sort_values('max_tile_mean', ascending=False)
    
    print("\n" + "="*60)
    print("ESTADÍSTICAS POR AGENTE")
    print("="*60)
    display(stats)
    
    # Guardar
    stats.to_csv('results/summary_statistics.csv')
    print("\n✓ Estadísticas guardadas en: results/summary_statistics.csv")

## 3. Visualización: Comparación de Max Tiles

Gráfico de barras comparando el max tile promedio alcanzado por cada agente.

In [ ]:
if df_all is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico 1: Max Tile Promedio
    agent_stats = df_all.groupby('agent_name')['max_tile'].agg(['mean', 'std']).sort_values('mean', ascending=False)
    
    ax1.bar(range(len(agent_stats)), agent_stats['mean'], 
            yerr=agent_stats['std'], capsize=5, alpha=0.7)
    ax1.set_xticks(range(len(agent_stats)))
    ax1.set_xticklabels(agent_stats.index, rotation=45, ha='right')
    ax1.set_ylabel('Max Tile Promedio')
    ax1.set_title('Comparación de Max Tile Alcanzado')
    ax1.grid(axis='y', alpha=0.3)
    
    # Añadir líneas de referencia
    for tile in [128, 256, 512, 1024, 2048]:
        ax1.axhline(y=tile, color='red', linestyle='--', alpha=0.3, linewidth=0.8)
        ax1.text(len(agent_stats)-0.5, tile, f'{tile}', fontsize=8, va='bottom')
    
    # Gráfico 2: Score Promedio
    score_stats = df_all.groupby('agent_name')['final_score'].mean().sort_values(ascending=False)
    
    ax2.bar(range(len(score_stats)), score_stats.values, alpha=0.7, color='orange')
    ax2.set_xticks(range(len(score_stats)))
    ax2.set_xticklabels(score_stats.index, rotation=45, ha='right')
    ax2.set_ylabel('Score Final Promedio')
    ax2.set_title('Comparación de Score Final')
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/plots/comparison_max_tile_score.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Gráfico guardado en: results/plots/comparison_max_tile_score.png")

## 4. Distribución de Max Tiles

Análisis de la distribución de fichas máximas alcanzadas.

In [ ]:
if df_all is not None:
    # Seleccionar top agentes para mejor visualización
    top_agents = df_all.groupby('agent_name')['max_tile'].mean().nlargest(10).index
    df_top = df_all[df_all['agent_name'].isin(top_agents)]
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Crear matriz de distribución
    distribution = pd.crosstab(df_top['agent_name'], df_top['max_tile'], normalize='index') * 100
    distribution = distribution.reindex(
        df_top.groupby('agent_name')['max_tile'].mean().sort_values(ascending=False).index
    )
    
    # Heatmap
    sns.heatmap(distribution, annot=True, fmt='.1f', cmap='YlOrRd', 
                cbar_kws={'label': 'Porcentaje (%)'}, ax=ax)
    ax.set_title('Distribución de Max Tiles Alcanzados (%)', fontsize=14, pad=20)
    ax.set_xlabel('Max Tile', fontsize=12)
    ax.set_ylabel('Agente', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('results/plots/max_tile_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Gráfico guardado en: results/plots/max_tile_distribution.png")

## 5. Análisis de Profundidad

Si hay experimentos con diferentes profundidades, analizar el impacto.

In [ ]:
if df_all is not None and 'depth' in df_all.columns:
    # Filtrar agentes con información de profundidad
    df_depth = df_all[df_all['depth'].notna()].copy()
    
    if len(df_depth) > 0:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Max Tile vs Depth
        depth_stats = df_depth.groupby('depth').agg({
            'max_tile': ['mean', 'std'],
            'final_score': 'mean',
            'time_seconds': 'mean',
            'moves': 'mean'
        })
        
        ax = axes[0, 0]
        depths = depth_stats.index
        ax.errorbar(depths, depth_stats['max_tile']['mean'], 
                   yerr=depth_stats['max_tile']['std'], 
                   marker='o', capsize=5, linewidth=2)
        ax.set_xlabel('Profundidad')
        ax.set_ylabel('Max Tile Promedio')
        ax.set_title('Impacto de la Profundidad en Max Tile')
        ax.grid(True, alpha=0.3)
        
        # 2. Tiempo vs Depth
        ax = axes[0, 1]
        ax.plot(depths, depth_stats['time_seconds']['mean'], 
               marker='s', linewidth=2, color='orange')
        ax.set_xlabel('Profundidad')
        ax.set_ylabel('Tiempo Promedio (segundos)')
        ax.set_title('Tiempo de Ejecución vs Profundidad')
        ax.grid(True, alpha=0.3)
        
        # 3. Score vs Depth
        ax = axes[1, 0]
        ax.plot(depths, depth_stats['final_score']['mean'], 
               marker='^', linewidth=2, color='green')
        ax.set_xlabel('Profundidad')
        ax.set_ylabel('Score Promedio')
        ax.set_title('Score vs Profundidad')
        ax.grid(True, alpha=0.3)
        
        # 4. Movimientos vs Depth
        ax = axes[1, 1]
        ax.plot(depths, depth_stats['moves']['mean'], 
               marker='d', linewidth=2, color='red')
        ax.set_xlabel('Profundidad')
        ax.set_ylabel('Número de Movimientos')
        ax.set_title('Duración del Juego vs Profundidad')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('results/plots/depth_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✓ Análisis de profundidad guardado en: results/plots/depth_analysis.png")
        
        # Tabla de comparación
        print("\n" + "="*60)
        print("COMPARACIÓN POR PROFUNDIDAD")
        print("="*60)
        display(depth_stats.round(2))

## 6. Análisis de Heurísticas

Comparación de diferentes configuraciones de heurísticas.

In [ ]:
if df_all is not None and 'heuristic_config' in df_all.columns:
    df_heur = df_all[df_all['heuristic_config'].notna()].copy()
    
    if len(df_heur) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Comparación de configuraciones
        heur_stats = df_heur.groupby('heuristic_config').agg({
            'max_tile': 'mean',
            'final_score': 'mean',
            'time_seconds': 'mean'
        }).sort_values('max_tile', ascending=False)
        
        # Gráfico 1: Max Tile
        ax = axes[0]
        heur_stats['max_tile'].plot(kind='barh', ax=ax, color='skyblue')
        ax.set_xlabel('Max Tile Promedio')
        ax.set_title('Comparación de Configuraciones Heurísticas')
        ax.grid(axis='x', alpha=0.3)
        
        # Gráfico 2: Tiempo vs Performance
        ax = axes[1]
        ax.scatter(heur_stats['time_seconds'], heur_stats['max_tile'], 
                  s=200, alpha=0.6, c=range(len(heur_stats)), cmap='viridis')
        
        for idx, config in enumerate(heur_stats.index):
            ax.annotate(config, 
                       (heur_stats.loc[config, 'time_seconds'], 
                        heur_stats.loc[config, 'max_tile']),
                       xytext=(5, 5), textcoords='offset points', fontsize=9)
        
        ax.set_xlabel('Tiempo Promedio (segundos)')
        ax.set_ylabel('Max Tile Promedio')
        ax.set_title('Trade-off: Tiempo vs Performance')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('results/plots/heuristic_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✓ Análisis de heurísticas guardado en: results/plots/heuristic_comparison.png")
        
        print("\n" + "="*60)
        print("RANKING DE HEURÍSTICAS")
        print("="*60)
        display(heur_stats.round(2))

## 7. Alpha-Beta Pruning Analysis

Análisis del impacto de Alpha-Beta Pruning en Minimax.

## 7. Alpha-Beta Pruning Analysis

Análisis del impacto de Alpha-Beta Pruning en Minimax.

## 8. Minimax vs Expectimax

Comparación directa entre los dos algoritmos principales.

In [ ]:
if df_all is not None:
    # Identificar agentes Minimax y Expectimax
    df_minimax = df_all[df_all['agent_name'].str.contains('Minimax', case=False, na=False)]
    df_expectimax = df_all[df_all['agent_name'].str.contains('Expectimax', case=False, na=False)]
    
    if len(df_minimax) > 0 and len(df_expectimax) > 0:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Distribución de Max Tiles
        ax = axes[0, 0]
        ax.hist([df_minimax['max_tile'], df_expectimax['max_tile']], 
               label=['Minimax', 'Expectimax'], bins=20, alpha=0.6)
        ax.set_xlabel('Max Tile')
        ax.set_ylabel('Frecuencia')
        ax.set_title('Distribución de Max Tiles')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 2. Box plot de Max Tile
        ax = axes[0, 1]
        data_to_plot = [df_minimax['max_tile'], df_expectimax['max_tile']]
        ax.boxplot(data_to_plot, labels=['Minimax', 'Expectimax'])
        ax.set_ylabel('Max Tile')
        ax.set_title('Comparación de Max Tile (Box Plot)')
        ax.grid(axis='y', alpha=0.3)
        
        # 3. Tiempo promedio
        ax = axes[1, 0]
        time_comparison = [df_minimax['time_seconds'].mean(), 
                          df_expectimax['time_seconds'].mean()]
        ax.bar(['Minimax', 'Expectimax'], time_comparison, 
              color=['coral', 'lightblue'], alpha=0.7)
        ax.set_ylabel('Tiempo Promedio (segundos)')
        ax.set_title('Tiempo de Ejecución')
        ax.grid(axis='y', alpha=0.3)
        
        # 4. Comparación de métricas
        ax = axes[1, 1]
        metrics_comp = pd.DataFrame({
            'Minimax': [
                df_minimax['max_tile'].mean(),
                df_minimax['final_score'].mean() / 1000,  # Escalar para visualización
                df_minimax['moves'].mean() / 10
            ],
            'Expectimax': [
                df_expectimax['max_tile'].mean(),
                df_expectimax['final_score'].mean() / 1000,
                df_expectimax['moves'].mean() / 10
            ]
        }, index=['Max Tile', 'Score (÷1000)', 'Moves (÷10)'])
        
        metrics_comp.plot(kind='bar', ax=ax, alpha=0.7)
        ax.set_title('Comparación de Métricas Normalizadas')
        ax.set_ylabel('Valor')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.savefig('results/plots/minimax_vs_expectimax.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✓ Comparación guardada en: results/plots/minimax_vs_expectimax.png")
        
        # Tabla comparativa
        print("\n" + "="*60)
        print("MINIMAX vs EXPECTIMAX - RESUMEN")
        print("="*60)
        
        comparison = pd.DataFrame({
            'Minimax': [
                df_minimax['max_tile'].mean(),
                df_minimax['max_tile'].std(),
                df_minimax['final_score'].mean(),
                df_minimax['moves'].mean(),
                df_minimax['time_seconds'].mean(),
                len(df_minimax)
            ],
            'Expectimax': [
                df_expectimax['max_tile'].mean(),
                df_expectimax['max_tile'].std(),
                df_expectimax['final_score'].mean(),
                df_expectimax['moves'].mean(),
                df_expectimax['time_seconds'].mean(),
                len(df_expectimax)
            ]
        }, index=['Max Tile (promedio)', 'Max Tile (std)', 'Score Final', 
                  'Movimientos', 'Tiempo (s)', 'Num Partidas'])
        
        display(comparison.round(2))

## 9. Mejor Agente y Recomendaciones

Identificación del mejor agente y conclusiones.

In [ ]:
if df_all is not None:
    print("\n" + "="*70)
    print(" "*20 + "🏆 MEJOR AGENTE 🏆")
    print("="*70 + "\n")
    
    # Rankear por max_tile promedio
    rankings = df_all.groupby('agent_name').agg({
        'max_tile': ['mean', 'std', 'max'],
        'final_score': 'mean',
        'moves': 'mean',
        'time_seconds': 'mean',
        'game_id': 'count'
    }).round(2)
    
    rankings.columns = ['_'.join(col).strip() for col in rankings.columns.values]
    rankings = rankings.sort_values('max_tile_mean', ascending=False)
    
    # Top 5
    print("TOP 5 AGENTES (por Max Tile promedio):\n")
    top5 = rankings.head(5)
    for idx, (agent, row) in enumerate(top5.iterrows(), 1):
        print(f"{idx}. {agent}")
        print(f"   Max Tile: {row['max_tile_mean']:.1f} (±{row['max_tile_std']:.1f})")
        print(f"   Mejor: {row['max_tile_max']:.0f}")
        print(f"   Score: {row['final_score_mean']:.0f}")
        print(f"   Tiempo: {row['time_seconds_mean']:.2f}s")
        print()
    
    # Identificar el mejor balance
    best_agent = rankings.index[0]
    print("="*70)
    print(f"✨ RECOMENDACIÓN: {best_agent}")
    print("="*70)
    
    best_stats = df_all[df_all['agent_name'] == best_agent]
    print(f"\nEstadísticas detalladas:")
    print(f"  - Partidas jugadas: {len(best_stats)}")
    print(f"  - Max Tile promedio: {best_stats['max_tile'].mean():.1f}")
    print(f"  - Probabilidad de 1024+: {(best_stats['max_tile'] >= 1024).sum() / len(best_stats) * 100:.1f}%")
    print(f"  - Probabilidad de 2048: {(best_stats['max_tile'] >= 2048).sum() / len(best_stats) * 100:.1f}%")
    print(f"  - Score promedio: {best_stats['final_score'].mean():.0f}")
    print(f"  - Tiempo promedio: {best_stats['time_seconds'].mean():.2f}s por partida")
    
    # Guardar configuración del mejor agente
    best_config = {
        'agent_name': best_agent,
        'stats': best_stats.describe().to_dict(),
        'timestamp': datetime.now().isoformat()
    }
    
    with open('models/best_agent_config.json', 'w') as f:
        json.dump(best_config, f, indent=2, default=str)
    
    print(f"\n✓ Configuración guardada en: models/best_agent_config.json")

## 10. Resumen Ejecutivo para el Informe

Genera un resumen con los hallazgos principales para incluir en el informe.

In [ ]:
if df_all is not None:
    print("\n" + "="*70)
    print(" "*15 + "📊 RESUMEN EJECUTIVO 📊")
    print("="*70 + "\n")
    
    # Hallazgos principales
    total_games = len(df_all)
    total_agents = df_all['agent_name'].nunique()
    total_time = df_all['time_seconds'].sum() / 3600  # en horas
    
    print(f"EXPERIMENTOS REALIZADOS:")
    print(f"  • Total de partidas: {total_games}")
    print(f"  • Agentes evaluados: {total_agents}")
    print(f"  • Tiempo total de computación: {total_time:.2f} horas")
    
    print(f"\nRESULTADOS GENERALES:")
    print(f"  • Max Tile promedio (todos los agentes): {df_all['max_tile'].mean():.1f}")
    print(f"  • Mejor max tile alcanzado: {df_all['max_tile'].max():.0f}")
    print(f"  • Partidas que alcanzaron 2048: {(df_all['max_tile'] >= 2048).sum()} ({(df_all['max_tile'] >= 2048).sum()/total_games*100:.1f}%)")
    
    # Comparación de algoritmos
    if 'Minimax' in df_all['agent_name'].str.cat() and 'Expectimax' in df_all['agent_name'].str.cat():
        minimax_avg = df_all[df_all['agent_name'].str.contains('Minimax', na=False)]['max_tile'].mean()
        expectimax_avg = df_all[df_all['agent_name'].str.contains('Expectimax', na=False)]['max_tile'].mean()
        
        print(f"\nCOMPARACIÓN MINIMAX vs EXPECTIMAX:")
        print(f"  • Minimax - Max Tile promedio: {minimax_avg:.1f}")
        print(f"  • Expectimax - Max Tile promedio: {expectimax_avg:.1f}")
        
        if expectimax_avg > minimax_avg:
            diff = ((expectimax_avg - minimax_avg) / minimax_avg) * 100
            print(f"  • Expectimax supera a Minimax en {diff:.1f}%")
            print(f"  ✓ Conclusión: Expectimax es más adecuado para juegos estocásticos")
        else:
            diff = ((minimax_avg - expectimax_avg) / expectimax_avg) * 100
            print(f"  • Minimax supera a Expectimax en {diff:.1f}%")
    
    # Impacto de profundidad
    if 'depth' in df_all.columns:
        depth_impact = df_all.groupby('depth')['max_tile'].mean()
        if len(depth_impact) > 1:
            print(f"\nIMPACTO DE LA PROFUNDIDAD:")
            for depth, avg_tile in depth_impact.items():
                print(f"  • Profundidad {depth}: {avg_tile:.1f}")
    
    # Mejores configuraciones
    print(f"\nCONFIGURACIONES ÓPTIMAS:")
    top3 = df_all.groupby('agent_name')['max_tile'].mean().nlargest(3)
    for i, (agent, score) in enumerate(top3.items(), 1):
        print(f"  {i}. {agent}: {score:.1f}")
    
    print("\n" + "="*70)
    print("✓ Este resumen puede ser incluido directamente en el informe")
    print("="*70)